In [1]:
import argparse
import numpy as np
import pickle
import time
import torch
import torch.nn as nn
import torch.optim as optim


from dataset import get_dataloader
from tqdm import tqdm
import random
from models.gated_tpp import *
import torch.nn.functional as F

In [2]:
data = '../data/simulated/sin_hawkes/'
batch_size = 32
def load_data(name, dict_name):
    with open(name, 'rb') as f:
        data = pickle.load(f, encoding='latin-1')
        num_types = data['dim_process']
        data = data[dict_name]
        return data, int(num_types)

train_data, num_types = load_data(data + 'train.pkl', 'train')
dev_data, _ = load_data(data + 'dev.pkl', 'dev')
test_data, _ = load_data(data + 'test.pkl', 'test')


trainloader = get_dataloader(train_data, 32, shuffle=True)
testloader = get_dataloader(test_data, 32, shuffle=False)
valloader = get_dataloader(dev_data, 32, shuffle=False)

In [3]:
for batch in testloader:
    pass

In [4]:
TPP = gated_TPP(1,16)
enc = Encoder(1,16,200)
h = TPP(batch[2],batch[0])


In [7]:
batch[1][0]

tensor([ 1.2200,  1.9236, 26.4183,  3.3898,  2.1395,  4.6247,  4.3740,  0.7937,
         0.3799,  0.3851,  7.2962,  4.2890,  4.8704,  0.9082,  2.6385,  3.3769,
         3.2499,  3.9915,  1.5987,  4.8717,  6.5974,  2.3238,  8.3797,  0.0403,
         1.6190,  2.4893,  2.2544,  0.5069,  1.9460,  0.2240,  0.7716,  3.3092,
         3.9283,  3.6769,  2.1405,  1.2271,  4.8555,  1.4609,  0.1759,  0.6158,
         1.6772,  2.3163,  2.2684,  1.1500,  0.3552,  0.3740,  1.8785,  0.9291,
         0.1469, 15.5452,  1.5675,  0.6699,  1.9982,  4.5895,  0.8847,  3.6575,
         2.2452,  2.3867,  6.4538,  9.0674,  3.3847,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000])